<a href="https://colab.research.google.com/github/TechWhizJyotirmoyee/Fine-tuning-SpeechT5-TTS-for-English-with-a-Focus-on-Technical-Vocabulary/blob/main/Fine_tuning_TTS_for_French_Language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Set Up Environment**

In [1]:
# Install Hugging Face Transformers and Datasets
!pip install transformers datasets

# For Bark, you can install it from GitHub if needed
!pip install git+https://github.com/suno-ai/bark.git

# Install PyTorch (ensure CUDA is available if you're using a GPU)
!pip install torch torchvision torchaudio

# Additional TTS tools
!pip install librosa soundfile numpy

# PyTorch Quantization
!pip install torch-quantization

# PyTorch Quantization
!pip install torch-quantization

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-7ktzqbtj
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-7ktzqbtj
  Resolved https://github.com/suno-ai/bark.git to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 43.8 MB/s eta 0:00:00
  Preparing metadat

**Dataset Preparation**

In [2]:
from datasets import load_dataset

# Load VoxPopuli dataset for your language, or use your own dataset
# Change: Using 'name' instead of 'language' and replacing "fr"
dataset = load_dataset("facebook/voxpopuli", name="fr", split="train")

# Example: Filtering dataset for a specific regional language
def filter_language(sample):
    #Instead of 'locale', use 'language' to filter
     return sample['language'] == 'fr' #'fr' for the main French language or specific for regions

filtered_dataset = dataset.filter(filter_language)

# Create a custom dataset with sentences for TTS fine-tuning
filtered_dataset = filtered_dataset.map(lambda sample: {"text": sample["transcript"], "audio": sample["audio_path"]})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


voxpopuli.py:   0%|          | 0.00/8.84k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

The repository for facebook/voxpopuli contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/voxpopuli.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


data/n_files.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

...

test_part_0.tar.gz:   0%|          | 0.00/605M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/73561 [00:00<?, ? examples/s]

**Fine-Tuning SpeechT5**

In [3]:
!pip install datasets

In [4]:
# Install Hugging Face Transformers and Datasets
!pip install transformers datasets

# For Bark, you can install it from GitHub if needed
!pip install git+https://github.com/suno-ai/bark.git

# Install PyTorch (ensure CUDA is available if you're using a GPU)
!pip install torch torchvision torchaudio

# Additional TTS tools
!pip install librosa soundfile numpy

# PyTorch Quantization
!pip install torch-quantization

# PyTorch Quantization
!pip install torch-quantization

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-txgr21xx
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-txgr21xx
  Resolved https://github.com/suno-ai/bark.git to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not find a version that satisfies the requirement torch-quantization (from versions: none)
ERROR: No matching distribution found for torch-quantization
ERROR: Could not find a version that satisfies the requirement torch-quantization (from versions: none)
ERROR: No matching distribution found for torch-quantization


In [8]:
from datasets import load_dataset

# Load VoxPopuli dataset for your language, or use your own dataset
# Change: Using 'name' instead of 'language' and replacing "fr"
dataset = load_dataset("facebook/voxpopuli", name="fr", split="train")

# Example: Filtering dataset for a specific regional language
def filter_language(sample):
    #Instead of 'locale', use 'language' to filter
    return sample['language'] == 'fr'  # 'fr' for the main French language or region-specific dialects

# Apply filtering to select only French samples
filtered_dataset = dataset.filter(filter_language)

# Reduce the dataset size to the first 100 samples (you can adjust the number as needed)
filtered_dataset = filtered_dataset.select(range(100))

# Create a custom dataset with sentences and audio paths for TTS fine-tuning
filtered_dataset = filtered_dataset.map(lambda sample: {"text": sample["transcript"], "audio": sample["audio_path"]})

# Print the first few samples before and after filtering
print("Original dataset sample:", dataset[:5])  # Original dataset sample
print("Filtered dataset sample (first 5):", filtered_dataset[:5])  # Filtered and reduced dataset sample


Original dataset sample: {'audio_id': ['20200212-0900-PLENARY-fr_20200212-18:11:25_1', '20170704-0900-PLENARY-20-fr_20170704-20:53:03_11', '20101124-0900-PLENARY-93-fr_20101124-22:03:52_2', '20120523-0900-PLENARY-3-fr_20120523-09:37:24_1', '20200710-0900-PLENARY-fr_20200710-09:56:28_8'], 'language': [2, 2, 2, 2, 2], 'audio': [{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c65191f3ea8bb55b8f233915dbb3bd232bda704e9fb9b1b3b78a87c2e35d6b4f/train_part_0/20200212-0900-PLENARY-fr_20200212-18:11:25_1.wav', 'array': array([ 3.66210938e-04, -9.15527344e-05, -2.74658203e-04, ...,
       -6.40869141e-04, -7.93457031e-04, -9.15527344e-04]), 'sampling_rate': 16000}, {'path': '/root/.cache/huggingface/datasets/downloads/extracted/c65191f3ea8bb55b8f233915dbb3bd232bda704e9fb9b1b3b78a87c2e35d6b4f/train_part_0/20170704-0900-PLENARY-20-fr_20170704-20:53:03_11.wav', 'array': array([ 6.10351562e-05, -2.13623047e-04,  1.25122070e-03, ...,
       -7.75146484e-03, -9.03320312e-03, -1.22680664e

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# ipython-input-7-a8bb45615161
from datasets import load_dataset

# Load VoxPopuli dataset for your language, or use your own dataset
# Change: Using 'name' instead of 'language' and replacing "fr"
dataset = load_dataset("facebook/voxpopuli", name="fr", split="train")

# Example: Filtering dataset for a specific regional language
def filter_language(sample):
    #Instead of 'locale', use 'language' to filter
     return sample['language'] == 'fr' #'fr' for the main French language or specific for regions

filtered_dataset = dataset.filter(filter_language)

# Create a custom dataset with sentences for TTS fine-tuning
filtered_dataset = filtered_dataset.map(lambda sample: {"text": sample["transcript"], "audio": sample["audio_path"]})


# ipython-input-1-a8bb45615161
# Print the first few samples before filtering
print("Original dataset sample:", dataset[:5]) # dataset is now defined and accessible

# Apply filtering
filtered_dataset = dataset.filter(filter_language)

# Print the first few samples after filtering
print("Filtered dataset sample:", filtered_dataset[:5])

Original dataset sample: {'audio_id': ['20200212-0900-PLENARY-fr_20200212-18:11:25_1', '20170704-0900-PLENARY-20-fr_20170704-20:53:03_11', '20101124-0900-PLENARY-93-fr_20101124-22:03:52_2', '20120523-0900-PLENARY-3-fr_20120523-09:37:24_1', '20200710-0900-PLENARY-fr_20200710-09:56:28_8'], 'language': [2, 2, 2, 2, 2], 'audio': [{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c65191f3ea8bb55b8f233915dbb3bd232bda704e9fb9b1b3b78a87c2e35d6b4f/train_part_0/20200212-0900-PLENARY-fr_20200212-18:11:25_1.wav', 'array': array([ 3.66210938e-04, -9.15527344e-05, -2.74658203e-04, ...,
       -6.40869141e-04, -7.93457031e-04, -9.15527344e-04]), 'sampling_rate': 16000}, {'path': '/root/.cache/huggingface/datasets/downloads/extracted/c65191f3ea8bb55b8f233915dbb3bd232bda704e9fb9b1b3b78a87c2e35d6b4f/train_part_0/20170704-0900-PLENARY-20-fr_20170704-20:53:03_11.wav', 'array': array([ 6.10351562e-05, -2.13623047e-04,  1.25122070e-03, ...,
       -7.75146484e-03, -9.03320312e-03, -1.22680664e

**Generate Audio**

In [14]:
import torch
from transformers import AutoProcessor, BarkModel  # Change this to SpeechT5 if you're using that

# Load your pre-trained model (change the model name accordingly)
processor = AutoProcessor.from_pretrained("suno/bark")  # For Bark
model = BarkModel.from_pretrained("suno/bark")  # For Bark

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load your dataset (make sure it's in the correct format)
from datasets import load_dataset

dataset = load_dataset("facebook/voxpopuli", "fr", split="train")

# Generate audio for the first entry in the dataset
text_to_synthesize = dataset[0]["raw_text"]

# Tokenize the input text
inputs = processor(text_to_synthesize, return_tensors="pt").to(device) # Move inputs to the same device as the model

# Generate audio
with torch.no_grad():
    audio = model.generate(**inputs)

# Save the generated audio to a file (WAV format)
import soundfile as sf

# Move the 'audio' tensor to the CPU before converting to NumPy
audio_cpu = audio.cpu()  # Add this line

# Assuming 'audio' is in the right format, e.g., float32
sf.write("generated_audio.wav", audio_cpu.squeeze().numpy(), 24000)  # Modify sample rate if needed

print("Audio generated and saved as 'generated_audio.wav'")

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


Audio generated and saved as 'generated_audio.wav'


**Quantization for Model Optimization**

In [6]:
import torch
import torch.quantization
from transformers import AutoProcessor, BarkModel

# Load your pre-trained model
processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")

# Set model to evaluation mode
model.eval()

# Remove weight normalization if it's applied to the model
def remove_weight_norm(model):
    for module in model.modules():
        try:
            torch.nn.utils.remove_weight_norm(module)
        except ValueError:
            # Some layers might not have weight_norm applied, ignore those
            pass

remove_weight_norm(model)

# Apply dynamic quantization directly to the original model
quantized_model = torch.quantization.quantize_dynamic(
    model,  # The original model instance to be quantized
    {torch.nn.Linear},  # Specify the layers to be quantized
    dtype=torch.qint8  # Quantization data type
)

# Save the quantized model
torch.save(quantized_model.state_dict(), "quantized_model.pth")

print("Model quantized and saved successfully.")


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


Model quantized and saved successfully.


In [8]:
import torch
from transformers import AutoProcessor, BarkModel
import IPython.display as ipd
import numpy as np
from datasets import load_dataset

# Load your pre-trained model and processor
processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")

# Set model to evaluation mode
model.eval()

# Load a sample dataset (this is just an example; replace with your dataset if needed)
dataset = load_dataset("facebook/voxpopuli", name="fr", split="train")

# Reduce the dataset size for quicker processing
dataset = dataset.select(range(5))  # Taking only first 5 samples as an example

# Function to generate audio from text input
def generate_audio(text):
    inputs = processor(text, return_tensors="pt")
    with torch.no_grad():
        audio_outputs = model.generate(**inputs)  # Generate the audio
    return audio_outputs[0].cpu().numpy()  # Convert to NumPy array for playback

# Example: Generate audio for the first text sample in the dataset
text_sample = dataset[0]["raw_text"]  # Replace with your own text if available
print(f"Generating audio for text: {text_sample}")

# Generate the audio
audio_array = generate_audio(text_sample)

# Convert audio to playable format (e.g., 16kHz, which is common for TTS)
sampling_rate = 16000  # Set the desired sampling rate (16kHz is typical for TTS models)
audio_normalized = np.int16(audio_array / np.max(np.abs(audio_array)) * 32767)  # Normalize audio

# Play the audio in the notebook
ipd.display(ipd.Audio(audio_normalized, rate=sampling_rate))

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a conseque

Generating audio for text: Notre délégation défendra la lutte contre les écarts salariaux à travail égal, contre les cyberviolences, les mariages forcés et les mutilations génitales.


**Evaluation**

In [3]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import torchaudio
import numpy as np
import os

# Load a pre-trained Wav2Vec2 model for evaluation
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Process audio samples for evaluation
def evaluate_speech(audio_file):
    # Check if the file exists
    if not os.path.exists(audio_file):
        raise FileNotFoundError(f"Audio file not found: {audio_file}")
    speech, rate = torchaudio.load(audio_file)
    input_values = processor(speech, sampling_rate=rate, return_tensors="pt").input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription

# Example: Generate synthetic speech, then evaluate
# Define generated_audio_files with a list of audio file paths
# Update the paths to your actual audio files
generated_audio_files = ["audio_1.wav", "audio_2.wav"]

for audio_file in generated_audio_files:
    try:
        transcription = evaluate_speech(audio_file)
        print(f"Transcription: {transcription}")
    except FileNotFoundError as e:
        print(e) # Print the error message if file is not found

# Manually rate MOS based on naturalness and intelligibility
mos_scores = [4.5, 4.0, 5.0, 3.5]  # Example MOS scores from different evaluators
average_mos = np.mean(mos_scores)
print(f"Average MOS score: {average_mos}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequen

Audio file not found: audio_1.wav
Audio file not found: audio_2.wav
Average MOS score: 4.25


**Final Steps: Benchmark Against Pre-trained Models**

In [5]:
# Compare inference speed, model size, and MOS score against a pre-trained SpeechT5 model
from transformers import SpeechT5ForTextToSpeech # Import the necessary class
pretrained_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
# Perform similar evaluation as above

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]